Role Title and Role Code are similar based on pandas
Instead of using hashing, can consider cantor(m,n) = 1/2 * (m+n)(m+n+1)+m. Beware of hash collision
PCA / SVD are more suited for continuous values. For discrete variables, consider using MCA
Step 1 : Generate 2 tuples / 3 tuples
Step 2 : Feature Extraction - Using Genetic Algorithm / Greedy Algorithm
Step 3 : Try using Stacked Classifier 
a) np.mean
b) weighted mean
c) jaccard score
d) pass the results from each classifer into another RF / Linear Regression



In [41]:
import pandas as pd
from pivottablejs import pivot_ui
import seaborn as sb
import matplotlib.pyplot as plt  
from ipywidgets import widgets
import numpy as np
from sklearn import (metrics, cross_validation, linear_model, preprocessing)
from sklearn.naive_bayes import BernoulliNB
from itertools import combinations

%matplotlib inline

sb.set(style="ticks", color_codes=True)
df = pd.read_csv('train.csv')
dft = pd.read_csv('test.csv')
#g = sb.pairplot(df)

In [194]:
df.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
#len(df['RESOURCE'].value_counts())
#df['RESOURCE'].plot.hist(bins=7518)

#dfc = df.iloc[:,1:10] 
dfc = df.iloc[:,:]
for col in df.columns:
  dfc[col] = df[col].astype('category')

dfc.describe()


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,32769,32769,32769,32769,32769,32769,32769,32769,32769,32769
unique,2,7518,4243,128,177,449,343,2358,67,343
top,1,4675,770,117961,118300,117878,118321,117906,290919,118322
freq,30872,839,152,21407,4424,1135,4649,6896,10980,4649


In [4]:
#test = pd.pivot_table(dfc,index=['ROLE_TITLE','ROLE_CODE'],aggfunc='count',dropna=True)
#dfc.pivot_table(index='ROLE_TITLE',aggfunc=lambda x:len(x.unique()))
mult = {}
for c in df.columns:
    temp = df.pivot_table(index=c,aggfunc=lambda x:len(x.unique()),fill_value=0).apply(np.max)
    mult[c] = temp

multpd = pd.DataFrame(mult)


In [3]:
def save_results(predictions, filename):
    """Given a vector of predictions, save results in CSV format."""
    with open(filename, 'w') as f:
        f.write("id,ACTION\n")
        for i, pred in enumerate(predictions):
            f.write("%d,%f\n" % (i + 1, pred))


In [8]:
# Convert from dataframe to np.array
y = np.ravel(df.iloc[:,:1])
X = df.iloc[:,1:9]
X_test = dft.iloc[:,1:9].values


In [40]:
# #Simplify the encoding. Probably not essential. Could possibly improve performance
# #Vertical Stacking is required so that all the features are captured. There might be some features in test set not in train set
# data_all = np.vstack((X,X_test))
# #data_all_t = data_all[1:10,:]
# le = preprocessing.LabelEncoder()
# data_t = []
# # To perform column wise operations, use Transpose function
# for i in data_all.T:
#     le.fit(i)
#     #Find the index for where test starts and train ends. 
#     data_test.append(le.transform(i[1000:]))
#     data_train.append(le.transform(i[1:1000])) 
# data_t = np.asarray(data_t).T
# print(data_t)
# #print(data_all_t)


[[3050 4440   21 ...,    4    7   65]
 [ 644  162   21 ...,   34   62   67]
 [2706 1679   50 ...,    0 2590    3]
 ..., 
 [3031 1042   21 ...,   19   41   65]
 [ 102 1403   21 ...,   22    7   65]
 [4090  555   21 ...,   75  866   14]]


In [38]:
# Get Combinations of 2 features    
data = np.vstack((df.iloc[:,1:-1],dft.iloc[:,1:-1]))
m,n = data.shape
print(m,n)

yc = combinations(range(n),2)
# to view all the combinations
list(yc) 

In [370]:
# #Code to help Visualize what the algorithm is doing 
# t_data = []
# test_data = np.array([[1,2,3,4],[1,2,2,0],[1,2,3,4],[1,4,5,6],[2,3,4,5]])
# print(test_data.shape)

# #if the combination of categories are the same, they will be hashed to the same value. Eg 1 row with (0,1,0) will have same 
# #hash value of another row with (0,1,0)
# for indices in combinations(range(4),2):
#     t_data.append(hash(tuple(v)) for v in test_data[:,indices])
#     #print(v for v in test_data[:,indices])
#     print(indices)
    
# #Comparison 1 - Combinations are (0,1),(0,2),(0,3),(1,2),(1,3),(2,3)
# # for v in test_data[:,(0,1)]:
# #   print(hash(tuple(v)))

# #Comparison 2 - t_data[0] corresponds to (0,1)
# #tuple(t_data[0])

(5, 4)
(0, 1)
(0, 2)
(0, 3)
(1, 2)
(1, 3)
(2, 3)


In [271]:
# #tuple is used here so that the result is hashable
# new_data = []
# for indices in combinations(range(n),2):
#     new_data.append(hash(tuple(v)) for v in data[:,indices])
#     #(print(v) for v in data.iloc[:,indices])


[[0 1 2 5 7 2 0 6]
 [5 5 4 4 1 0 4 1]
 [4 3 2 5 5 1 3 5]
 [6 4 0 0 4 5 2 2]
 [7 6 1 1 2 3 1 1]
 [2 7 2 5 8 4 6 3]
 [1 2 2 2 3 6 5 4]
 [3 0 2 6 6 7 7 0]
 [8 8 3 3 0 0 8 1]]
[[ 17183   1540 117961 118343 123125 118536 118536 308574]
 [ 36724  14457 118219 118220 117884 117879 267952  19721]
 [ 36135   5396 117961 118343 119993 118321 240983 290919]
 [ 42680   5905 117929 117930 119569 119323 123932  19793]
 [ 45333  14561 117951 117952 118008 118568 118568  19721]
 [ 25993  17227 117961 118343 123476 118980 301534 118295]
 [ 19666   4209 117961 117969 118910 126820 269034 118638]
 [ 31246    783 117961 118413 120584 128230 302830   4673]
 [ 78766  56683 118079 118080 117878 117879 304519  19721]]


In [197]:
encoder = preprocessing.OneHotEncoder()
# we want to encode the category IDs encountered both in
# the training and the test set, so we fit the encoder on both
encoder.fit(np.vstack((X,X_test)))
X = encoder.transform(X)
X_test = encoder.transform(X_test)

In [198]:
print(encoder.n_values_)
print(encoder.feature_indices_)

[312154 311780 311179 286792 286793 311868 311868 308575]
[      0  312154  623934  935113 1221905 1508698 1820566 2132434 2441009]


In [191]:
SEED = 42
mean_auc = 0.0

n = 10  # repeat the CV procedure 10 times to get more precise results
#model = linear_model.LogisticRegression(C=1)
model = BernoulliNB(alpha=0.005)
for i in range(n):
    # for each iteration, randomly hold out 20% of the data as CV set
    X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(X, y, test_size=.20, random_state=i*SEED)

    # if you want to perform feature selection / hyperparameter
    # optimization, this is where you want to do it

    # train model and make predictions
    model.fit(X_train, y_train) 
    preds = model.predict_proba(X_cv)[:, 1]

    # compute AUC metric for this CV fold
    fpr, tpr, thresholds = metrics.roc_curve(y_cv, preds)
    roc_auc = metrics.auc(fpr, tpr)
    print("AUC (fold %d/%d): %f" % (i + 1, n, roc_auc))
    mean_auc += roc_auc

print("Mean AUC: %f" % (mean_auc/n))

# === Predictions === #
# When making predictions, retrain the model on the whole training set
model.fit(X, y)
preds = model.predict_proba(X_test)[:, 1]
save_results(preds, "submit.csv")

AUC (fold 1/10): 0.840287
AUC (fold 2/10): 0.837616
AUC (fold 3/10): 0.838619
AUC (fold 4/10): 0.843097
AUC (fold 5/10): 0.823374
AUC (fold 6/10): 0.848091
AUC (fold 7/10): 0.814234
AUC (fold 8/10): 0.829486
AUC (fold 9/10): 0.820575
AUC (fold 10/10): 0.839104
Mean AUC: 0.833448


In [378]:
# Numpy Operations
# print(np.fromiter(t_data[0],np.int32))
# print(np.fromiter(t_data[1],np.int32))
# print(np.fromiter(t_data[2],np.int32))
# print(np.fromiter(t_data[3],np.int32))
# print(np.fromiter(t_data[4],np.int32))
# print(np.fromiter(t_data[5],np.int32))

# Convert to numpy array - Use np.array(list)
# for i in nparray - iterate through rows
# for i in nparray.T - iterate through columns
# Initialize np.zeros(5)

# To view generator operators (Use list,tuples)
# for i,j in enumerate(t_data):
#     #lst_array[i] = np.array(list(j))
#     lst_array[i] = np.fromiter(j,np.float)

AUC = Logistic Regression returns the probability that a certain input features would be in a certain class
ROC curve visualizaes all possible thresholds
Misclassification rate is error rate for a single threshold